In [1]:
%%capture

!pip install -U "transformers>=4.42.3" 
!pip install -U "bitsandbytes>0.42.1" 
!pip install accelerate peft

Load model and add LoRA weights:

In [2]:
import time
from dataclasses import dataclass

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    PreTrainedTokenizerBase
)
from peft import PeftModel

@dataclass
class Config:
    model_name = "/kaggle/input/mistral-nemo-12b-4-bit/transformers/instruction-tuned/1/Mistral-Nemo-Instruct-12b-4bit"
    lora_dir = "/kaggle/input/20q-mistral-nemo-lora-adapters/transformers/default/4/lora_adapters_final_600_games/checkpoint-7437"
    max_length = 2048
    batch_size = 1
    device = torch.device("cuda")    
    tta = False  
    spread_max_length = False  

cfg = Config()

tokenizer = AutoTokenizer.from_pretrained(cfg.model_name, add_bos_token = True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quanty_type = "fp4", 
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quanty = True,
)

model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name,
    quantization_config = bnb_config,
    torch_dtype = torch.float16,
    device_map = "auto",
    attn_implementation="eager",
    trust_remote_code = True,
)

model = PeftModel.from_pretrained(model, cfg.lora_dir)

2024-08-08 19:53:56.801558: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 19:53:56.801758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 19:53:56.929750: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Unused kwargs: ['bnb_4bit_quanty_type', 'bnb_4bit_use_double_quanty']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/opt/conda/lib/python3.10/site-

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Functions to format a game (class obs) for the model:

In [3]:
class obs(object):
    def __init__(self, keyword, questions, answers, turnType, category):
        self.turnType = turnType
        self.keyword = keyword
        self.category = category
        self.questions = questions
        self.answers = answers

import re

class MistralPromptFormatter:
    def __init__(self, inference_mode = True):
        self.sys_prompt = "You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. "\
        "The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. "\
        "Then the Guesser tries to guess the keyword based on the questions and answers in the game.\n"
        self.inference_mode = inference_mode
        
    def format_instruction(self, instruction):
        return f"[INST]{instruction}[/INST]"
    
    def format_obs(self, obs, guesses):
        if obs.turnType == 'ask':
            return self.ask_format(obs,guesses)
        if obs.turnType == 'answer':
            return self.answer_format(obs,guesses)
        if obs.turnType == 'guess':
            return self.guess_format(obs, guesses)
    
    def game_to_string(self, obs):
        game_string = ""
        for question, answer in zip(obs.questions, obs.answers):
            game_string += f"Question: {question}\nAnswer:{answer}\n"
        print(game_string)
        return game_string
        
    def summarizer_format(self, obs):
        sum_prompt = ("You are an AI assistant for the 20 Questions game. In this game the Answerer is given a secret keyword. "
                      "The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. "
                      "Then the Guesser tries to guess the keyword based on the questions and answers in the game. "
                      "You are assisting the game by summarizing the information the Questioner has obtained about the keyword from his questions and the Answerer's answers. "
                      "When prompted, read all questions and answers that are given to you and give a straightforward and concise summary of the information in the game so far. "
                      "The keyword is a specific thing, NOT a place and NOT a person. \n\n"
                      "Here are all the questions and answers regarding the keyword in the game so far:\n" + self.game_to_string(obs) + "\nNow write a summary of what is known "
                      "about the keyword based on the ALL questions and answers above. Include information from all questions and answers above."
                      "\nOpt to say the keyword is a thing, as opposed to an object.\n Summary:")
        formatted_conversation = self.format_instruction(sum_prompt)
        return formatted_conversation
        
    def ask_format(self, obs, guesses):
        ask_prompt = ("You are playing this game as the Questioner. Ask yes-or-no questions that narrow down what the keyword could be. The keyword is a specific thing, NOT a place and NOT a person. "
                      "DO NOT ask if the keyword is a specific thing, rather ask something about the keyword, as in the following examples:\n"
                      "Example 1: Do NOT ask: 'Is the keyword knife?', INSTEAD ask: 'Is it a knife used for a particular purpose?'\n"
                      "Example 2: Do NOT ask: 'Is the keyword cow?', INSTEAD ask: 'Is it a specific type or breed of cow?'\n"
                      "Example 3: Do NOT ask: 'Is it bottle?', INSTEAD ask: 'Is it a bottle made of a specific material?'\n"
                      "Example 4: Do NOT ask: 'Is the keyword lamp?', INSTEAD ask: 'Is it a type of lamp?'\n"
                      "Do NOT assume the game has ended, the game will determine when to stop. Instead keep asking questions about the keyword. "
                      "Do not output any text other than the question. Start with broad questions "
                      "and based on the answers so far narrow down your questions. Now ask your first question.\n\nQuestion: ")
        formatted_conversation = self.format_instruction(self.sys_prompt + ask_prompt) 
        for i in range(len(obs.questions)):
            formatted_conversation += obs.questions[i] + "</s>"
            if self.inference_mode or i != len(obs.questions)-1: 
                formatted_conversation += '\n' + self.format_instruction(f"Answer: {obs.answers[i]}\nAsk your next question.\nQuestion:")  
        return formatted_conversation.strip()
    
    def guess_format(self, obs, guesses):
        guess_prompt = "You are playing this game as the Guesser. After each question and answer you will guess what the keyword is based on the knowledge you have gained "\
        "from the questions and answers about the keyword. Do NOT give repeated guesses.\n\nRound 1:\nQuestion: " + obs.questions[0]+'\nAnswer: '+ obs.answers[0] +".\nNow guess the keyword."
        formatted_conversation = self.format_instruction(self.sys_prompt + guess_prompt) 
        for i in range(len(guesses)):
            formatted_conversation += guesses[i] + "</s>"
            if self.inference_mode or i < len(guesses)-1: 
                formatted_conversation += '\n' + self.format_instruction(f"Round {i+2}: \nQuestion: "+obs.questions[i+1]+"\nAnswer: "+obs.answers[i+1]+".\n"
                                                                         "Now guess the keyword based on the information from all rounds about the keyword. Do not give repeated guesses.\nGuess:")  
        return formatted_conversation.strip()
    
    def answer_format(self, obs, guesses):
        answer_prompt = "You are playing this game as the Answerer. You will answer accurately the questions regarding the keyword with ONLY yes or no. "\
        "If the questioner did not ask a question, simply answer 'yes'. "\
        f"For this game the keyword is {obs.keyword}. Now answer the question about the keyword.\nKeyword: {obs.keyword}.\nQuestion: " + obs.questions[0] +"\nAnswer:"
        formatted_conversation = self.format_instruction(self.sys_prompt + answer_prompt) 
        for i in range(len(obs.answers)):
            formatted_conversation += obs.answers[i] + "</s>"
            if i != len(obs.questions)-1: 
                formatted_conversation += '\n' + self.format_instruction(f"Answer the following question about the keyword.\nKeyword: {obs.keyword}. \nQuestion: " + obs.questions[i+1]+"\nAnswer:")  
        return formatted_conversation.strip()
    
    def format_response(self, string, string_len, obs):
        response = string[string_len+3:]
        response = response.replace('</s>', '')
        if obs.turnType in ['guess', 'ask']:
            return response
        else:
            response = response.lower()
            if 'yes' in response: 
                return "yes"
            elif 'no' in response: 
                return "no"
            else: 
                return "yes"
            
    def get_guess_variants(self, guess):
        trimmed_string = guess.lstrip()
        if not trimmed_string:
            return []
        if not trimmed_string[0].isalpha():
            return []
        uppercase_version = trimmed_string.capitalize()
        lowercase_version = trimmed_string[0].lower() + trimmed_string[1:]  
        return [uppercase_version, lowercase_version]

formatter = MistralPromptFormatter(inference_mode = True)


#################
# Define main function to call model and generate response

def get_words_ids(words):
    return [tokenizer.encode(word, add_special_tokens=False) for word in words]

def tokenize(game_text, tokenizer):
    return tokenizer(game_text, return_tensors="pt")

def generate_response(obs, guesses):
    game_text = formatter.format_obs(obs, guesses)
    game_text_len = len(game_text)
    game_tokens = tokenize(game_text, tokenizer)
    output_tokens = model.generate(input_ids=game_tokens["input_ids"].to("cuda"),
                                   attention_mask=game_tokens["attention_mask"].to("cuda"),
                                   pad_token_id = 2,
                                   max_new_tokens=64)
    output = tokenizer.batch_decode(output_tokens.detach().cpu().numpy(), skip_special_tokens=False)[0]  
    output = formatter.format_response(output, game_text_len, obs) 
    if obs.turnType == 'guess':
        guesses.append(output)
    return output

def generate_summary(obs):
    game_text = formatter.summarizer_format(obs)
    game_text_len = len(game_text)
    game_tokens = tokenize(game_text, tokenizer)
    output_tokens = model.generate(input_ids=game_tokens["input_ids"].to("cuda"),
                                   attention_mask=game_tokens["attention_mask"].to("cuda"),
                                   pad_token_id = 2,
                                   max_new_tokens=256)
    output = tokenizer.batch_decode(output_tokens.detach().cpu().numpy(), skip_special_tokens=False)[0]  
    output = formatter.format_response(output, game_text_len, obs) 
    return output

#################
# Function to be called by the game environment

def agent_fn(obs, cfg, guesses):
    response = generate_response(obs, guesses)
    #Catch glitched responses
    if (response is None) or (len(response) <= 1) or (len(response) >= 200):
        return "yes"    
    else:
        return response

Choose an example game by modifying the cell below. Turn types are 'ask', 'guess' and 'answer'.

In [4]:
test_game = obs(
    turnType = 'guess',
    keyword = 'klein bottle',
    category = 'thing',
    questions = ['Is it a place?', 
                 'Is it broadly related to food, drinks or cooking?',
                 'Is it broadly related to electronics or technology?',
                 'Is it broadly related to arts, sports or entertainment?',
                 'Is it broadly related to clothing, beauty or accessories?',
                 'Is it man-made?',
                 'Is it found indoors?',
                 'Is it broadly related to furniture or architecture?'
                ],
    answers = ['no', 'no', 'no', 'no','no','yes','yes','yes']
)

test_guesses = ['chair', 'book', 'pen', 'clock', 'table', 'lamp', 'sofa']

In [5]:
summary = generate_summary(test_game)
print(f"\nSummary: {summary}")

Question: Is it a place?
Answer:no
Question: Is it broadly related to food, drinks or cooking?
Answer:no
Question: Is it broadly related to electronics or technology?
Answer:no
Question: Is it broadly related to arts, sports or entertainment?
Answer:no
Question: Is it broadly related to clothing, beauty or accessories?
Answer:no
Question: Is it man-made?
Answer:yes
Question: Is it found indoors?
Answer:yes
Question: Is it broadly related to furniture or architecture?
Answer:yes


Summary: The keyword is a type of furniture. It is primarily used for sitting or lying down. It is typically found in a living room or bedroom. It is a piece of furniture that is commonly found in households.


In [6]:
print(summary)

The keyword is a type of furniture. It is primarily used for sitting or lying down. It is typically found in a living room or bedroom. It is a piece of furniture that is commonly found in households.


Run this cell to generate a response:

In [7]:
response = generate_response(test_game, test_guesses)
formatter_print = MistralPromptFormatter(inference_mode = False)

print("----- Input -----")
print(formatter_print.format_obs(test_game, test_guesses))
print("\n----- Model Response -----")
print(response)

----- Input -----
[INST]You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. Then the Guesser tries to guess the keyword based on the questions and answers in the game.
You are playing this game as the Guesser. After each question and answer you will guess what the keyword is based on the knowledge you have gained from the questions and answers about the keyword. Do NOT give repeated guesses.

Round 1:
Question: Is it a place?
Answer: no.
Now guess the keyword.[/INST]chair</s>
[INST]Round 2: 
Question: Is it broadly related to food, drinks or cooking?
Answer: no.
Now guess the keyword based on the information from all rounds about the keyword. Do not give repeated guesses.
Guess:[/INST]book</s>
[INST]Round 3: 
Question: Is it broadly related to electronics or technology?
Answer: no.
Now guess the keyword based on the inform

In [8]:
def simulate_game(keyword):
    curr_guesses = []
    curr_game = obs(keyword = keyword, questions = [], answers = [], turnType = 'ask', category = None)
    for turn in range(20):
        curr_game.turnType = 'ask'
        question = agent_fn(curr_game, None, curr_guesses)
        print(f"Question {turn+1}: {question}")
        curr_game.questions.append(question)
        
        curr_game.turnType = 'answer'
        answer = agent_fn(curr_game, None, curr_guesses)
        print(f"Answer {turn+1}: {answer}")
        curr_game.answers.append(answer)
        
        curr_game.turnType = 'guess'
        guess = agent_fn(curr_game, None, curr_guesses)
        print(f"Guess {turn+1}: {guess}\n")
    return

In [9]:
simulate_game('trophy')

Question 1: Is it related to food, beverages or cooking?
Answer 1: no
Guess 1: Car

Question 2: Is it related to industrial production or manufacturing?
Answer 2: no
Guess 2: Tree

Question 3: Is it broadly related to entertainment or sports?
Answer 3: yes
Guess 3: Television

Question 4: Is it something a person can hold in their hand?
Answer 4: yes
Guess 4: Playing card

Question 5: Is it related to electronics or technology?
Answer 5: no
Guess 5: Book

Question 6: Is it broadly related to the arts or media?
Answer 6: no
Guess 6: Board game

Question 7: Is it used in a specific sport or recreational activity?
Answer 7: yes
Guess 7: Tennis ball

Question 8: Is it used in a team sport?
Answer 8: yes
Guess 8: Soccer ball

Question 9: Is it a type of ball?
Answer 9: no
Guess 9: Hockey stick

Question 10: Is it protective equipment?
Answer 10: no
Guess 10: Whistle

Question 11: Is it used to score points or goals in a game?
Answer 11: no
Guess 11: Referee flag

Question 12: Is it a type o

In [10]:
from kaggle_environments import make
import kaggle_environments
import random

def random_agent(obs, cfg):
    if obs.turnType == "ask": response = "Is it a duck?"
    elif obs.turnType == "guess": response = "duck"
    elif obs.turnType == "answer": 
        random_int = random.randint(0, 1)
        if random_int: response = "Yes"
        else: response = "No"
    return response

class obs(object):
    def __init__(self, turnType, keyword, category, questions, answers):
        self.turnType = turnType
        self.keyword = keyword
        self.category = category
        self.questions = questions
        self.answers = answers
        
keyword = "argentina"
alts = ["argentina"]
kaggle_environments.envs.llm_20_questions.llm_20_questions.category = "Place"
kaggle_environments.envs.llm_20_questions.llm_20_questions.keyword_obj = {'keyword':keyword,'alts':alts}
kaggle_environments.envs.llm_20_questions.llm_20_questions.keyword = keyword
kaggle_environments.envs.llm_20_questions.llm_20_questions.alts = alts

#env = make("llm_20_questions", debug=True)
#game_output = env.run(agents=[agent_fn, random_agent, random_agent, random_agent])
#env.render(mode="ipython", width=800, height=400)

No pygame installed, ignoring import
